In [2]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from nltk.tokenize import sent_tokenize

In [3]:
df = pd.read_csv("/Users/rubing/Documents/Donnees/covax_content_sample_2000_medium_sized.csv").drop("Unnamed: 0", axis=1)

In [45]:
TEXT=df.extracted_text[2]
print(TEXT)

People who have gotten the new coronavirus and recovered can get it again in the future, health authorities say — the body does not become immune after infection.
On Wednesday, Japanese authorities reported the first confirmed case of reinfection. A tour guide in Osaka first tested positive for the coronavirus in late January, then was discharged from the hospital three weeks ago after showing signs of recovery. But she returned to the hospital after developing a sore throat and chest pain and tested positive for the coronavirus once again.
Zhan Qingyuan, director of pneumonia prevention and treatment at the China-Japan Friendship Hospital, warned last month that this could  happen.
"For those patients who have been cured, there is a likelihood of a relapse," Zhan said in a briefing on January 31 . "The antibody will be generated; however, in certain individuals, the antibody cannot last that long."
Reinfections among patients in China have been reported as well.
In total, the coronavi

In [5]:
import fr_core_news_md

nlp = fr_core_news_md.load()

In [58]:
import en_core_web_md

nlp = en_core_web_md.load()

In [18]:
docs = nlp.pipe(texts)

In [10]:
def seg_doc(text):
    #V2 : regrouper les enfants isolés de la racine avec la racine

    doc = nlp(text)       #pour traiter un ensemble de documents, utiliser docs =nlp.pipe(DOCS)

    
    segments = []
    roots = [token for token in doc if token.head == token]
    for root in roots: 
        seg_root = []
        for token in root.lefts:                  #segmenter en sous-arbre
            if (len(list(token.subtree)) == 1) & (token.pos_ not in ['SPACE','PUNCT']):
                seg_root.append(token)
            else: 
                temp = []
                for descendant in token.subtree:            
                    if (descendant.text == ",") & (len(temp) != 0):        #segmenter en virgule
                        segments.append(temp)
                        temp = []
                    else:
                        if descendant.pos_ not in ['SPACE','PUNCT']:
                            temp.append(descendant)
                if len(temp) != 0:
                    segments.append(temp)
        position_root = len(segments)            
        seg_root.append(root)
        
        for token in root.rights:
            if (len(list(token.subtree)) == 1) & (token.pos_ not in ['SPACE','PUNCT']):
                seg_root.append(token)
            else: 
                temp = []
                for descendant in token.subtree:            
                    if (descendant.text == ",") & (len(temp) != 0):        
                        segments.append(temp)
                        temp = []
                    else:
                        if descendant.pos_ not in ['SPACE','PUNCT']:
                            temp.append(descendant)
                if len(temp) != 0:
                    segments.append(temp)
        segments.insert(position_root, seg_root)            

    return segments

Texte 2 (anglais), phrases 0, 4

In [75]:
i = 5

SENT = sent_tokenize(TEXT)[i]
print(SENT)

"For those patients who have been cured, there is a likelihood of a relapse," Zhan said in a briefing on January 31 .


In [76]:
seg_doc(SENT)

[[For, those, patients, who, have, been, cured],
 [there, is, a, likelihood, of, a, relapse],
 [Zhan, said],
 [in, a, briefing, on, January, 31]]

In [77]:
displacy.render(nlp(SENT), style="dep")

Exploration : sociological tokenizer

Seperate 1 sentence in three listes : source, predicat, token.

In [10]:
print(TEXT)

Un million de tests pourraient être réalisés d'ici la fin de la semaine. Et un traitement pourrait être disponible d'ici l'été ou l'automne, selon Mike Pence.
Aux États-Unis aussi, les morts se multiplient. Avec quatre nouveaux décès enregistrés lundi, six personnes ont succombé au coronavirus outre-Atlantique, et la barre des cent contaminés a été atteinte. Selon le New York Times , l'administration Trump a alors annoncé qu' un million de tests pourraient être réalisés d'ici la fin de la semaine. Des compagnies privées et des laboratoires académiques ont été poussés à développer et valider leur propre dépistage.  
LIRE AUSSI >> Coronavirus : les Américains sans couverture santé dans l'impasse  
D'autre part, un traitement médical contre le nouveau coronavirus pourrait être disponible "d'ici l'été ou le début de l'automne", a affirmé lundi le vice-président américain Mike Pence , qui coordonne la lutte contre l'épidémie aux États-Unis.  Offre limitée. 2 mois pour 1€ sans engagement Je 

In [16]:
seg_doc(TEXT)[6][0].pos_   

'PUNCT'

Redécouper de manière tant qu'il y a verbe

In [20]:
def has_verb(doc):
    has_verb = False
    for token in doc:
        if token.pos_ in ["VERB", "AUX"]:
            has_verb = True
            break
    return has_verb

In [25]:
def seg2sent(seg):
    phrase = ""
    for token in seg:
        phrase += token.text + " " 
    return phrase

In [27]:
#version hiérarchique

def seg_doc_2(segments):
    for i,seg in enumerate(segments):
        if has_verb(seg):
            new_segs = seg_doc(seg2sent(seg))
            segments[i] = new_segs
        else:
            segments[i] = [segments[i]]
    return segments
            

In [28]:
print(TEXT)

Un million de tests pourraient être réalisés d'ici la fin de la semaine. Et un traitement pourrait être disponible d'ici l'été ou l'automne, selon Mike Pence.
Aux États-Unis aussi, les morts se multiplient. Avec quatre nouveaux décès enregistrés lundi, six personnes ont succombé au coronavirus outre-Atlantique, et la barre des cent contaminés a été atteinte. Selon le New York Times , l'administration Trump a alors annoncé qu' un million de tests pourraient être réalisés d'ici la fin de la semaine. Des compagnies privées et des laboratoires académiques ont été poussés à développer et valider leur propre dépistage.  
LIRE AUSSI >> Coronavirus : les Américains sans couverture santé dans l'impasse  
D'autre part, un traitement médical contre le nouveau coronavirus pourrait être disponible "d'ici l'été ou le début de l'automne", a affirmé lundi le vice-président américain Mike Pence , qui coordonne la lutte contre l'épidémie aux États-Unis.  Offre limitée. 2 mois pour 1€ sans engagement Je 

In [94]:
SG = seg_doc(TEXT)

In [95]:
SG

[[Un, million, de, tests],
 [être, réalisés, d', ici, la, fin, de, la, semaine],
 [pourraient, .],
 [un, traitement],
 [être, disponible, d', ici, l', été, ou, l', automne, ,],
 [selon, Mike, Pence],
 [., ],
 [Et, pourrait],
 [Aux, États-Unis],
 [les, morts],
 [aussi, ,, se, multiplient, .],
 [Avec, quatre, nouveaux, décès, enregistrés, lundi],
 [six, personnes],
 [au, coronavirus, outre-Atlantique],
 [et, la, barre, des, cent, contaminés, a, été, atteinte],
 [,, ont, succombé, ,, .],
 [Selon, le, New, York, Times],
 [l', administration, Trump],
 [qu,
  ',
  un,
  million,
  de,
  tests,
  pourraient,
  être,
  réalisés,
  d',
  ici,
  la,
  fin,
  de,
  la,
  semaine],
 [,, a, alors, annoncé, .],
 [Des, compagnies, privées, et, des, laboratoires, académiques],
 [à, développer, et, valider, leur, propre, dépistage],
 [.,  ],
 [ont, été, poussés],
 [LIRE, AUSSI],
 [>, >],
 [les, Américains, sans, couverture, santé, dans, l', impasse,  ],
 [Coronavirus, :],
 [D', autre, part, ,],
 [un, t

In [96]:
seg_doc_2(SG)

[[[Un, million, de, tests]],
 [[d, ', ici, la, fin, de, la, semaine], [être, réalisés]],
 [[pourraient, .]],
 [[un, traitement]],
 [[être, disponible], [ici, l], [été, ou, l, '], [d, ', ', automne, ,]],
 [[selon, Mike, Pence]],
 [[., ]],
 [[Et, pourrait]],
 [[Aux, États-Unis]],
 [[les, morts]],
 [[aussi, ,, se, multiplient, .]],
 [[enregistrés, lundi], [Avec, quatre, nouveaux, décès]],
 [[six, personnes]],
 [[au, coronavirus, outre-Atlantique]],
 [[et, la, barre, des, cent, contaminés], [a, été, atteinte]],
 [[,, ont, succombé, ,, .]],
 [[Selon, le, New, York, Times]],
 [[l', administration, Trump]],
 [[un, million, de, tests],
  [être, réalisés, d, ', ici, la, fin, de, la, semaine],
  [qu, ', pourraient]],
 [[,, a, alors, annoncé, .]],
 [[Des, compagnies, privées, et, des, laboratoires, académiques]],
 [[et, valider, leur, propre, dépistage], [à, développer]],
 [[.,  ]],
 [[ont, été, poussés]],
 [[LIRE, AUSSI]],
 [[>, >]],
 [[les, Américains, sans, couverture, santé, dans, l', impasse

In [97]:
for segments in SG:
    segments = seg_doc_2(segments)

In [98]:
for segseg in SG:
    for seg in segseg:
        seg = seg_doc_2(seg)

In [99]:
SG

[[[[[Un, million, de, tests]]]],
 [[[[d, ', ici, la, fin, de, la, semaine]]], [[[être, réalisés]]]],
 [[[[pourraient, .]]]],
 [[[[un, traitement]]]],
 [[[[être, disponible]]],
  [[[ici, l]]],
  [[[ou, l]], [[été, ']]],
  [[[d, ', ', automne, ,]]]],
 [[[[selon, Mike, Pence]]]],
 [[[[., ]]]],
 [[[[Et, pourrait]]]],
 [[[[Aux, États-Unis]]]],
 [[[[les, morts]]]],
 [[[[aussi, ,, se, multiplient, .]]]],
 [[[[enregistrés, lundi]]], [[[Avec, quatre, nouveaux, décès]]]],
 [[[[six, personnes]]]],
 [[[[au, coronavirus, outre-Atlantique]]]],
 [[[[et, la, barre, des, cent, contaminés]]], [[[a, été, atteinte]]]],
 [[[[,, ont, succombé, ,, .]]]],
 [[[[Selon, le, New, York, Times]]]],
 [[[[l', administration, Trump]]]],
 [[[[un, million, de, tests]]],
  [[[d, ', ici, la, fin, de, la, semaine]], [[être, réalisés]]],
  [[[qu, ', pourraient]]]],
 [[[[,, a, alors, annoncé, .]]]],
 [[[[Des, compagnies, privées, et, des, laboratoires, académiques]]]],
 [[[[leur, propre, dépistage]], [[et, valider]]], [[[à, 

# Version 2

In [150]:
#version flatten

def seg_doc_2(segments):
    temp = [seg for seg in segments]
    for i,seg in enumerate(segments):
        if has_verb(seg):
            new_segs = seg_doc(seg2sent(seg))
            temp[i] = new_segs
        else:
            temp[i] = [segments[i]]
    return [seg for segs in temp for seg in segs]

In [148]:
def seg_auto(TEXT):
    segments = seg_doc(TEXT)
    c = 0
    while len(seg_doc_2(segments)) > len (segments):
        segments = seg_doc_2(segments)
        c += 1
        print(c)
    return segments

In [157]:
print(TEXT)

Un million de tests pourraient être réalisés d'ici la fin de la semaine. Et un traitement pourrait être disponible d'ici l'été ou l'automne, selon Mike Pence.
Aux États-Unis aussi, les morts se multiplient. Avec quatre nouveaux décès enregistrés lundi, six personnes ont succombé au coronavirus outre-Atlantique, et la barre des cent contaminés a été atteinte. Selon le New York Times , l'administration Trump a alors annoncé qu' un million de tests pourraient être réalisés d'ici la fin de la semaine. Des compagnies privées et des laboratoires académiques ont été poussés à développer et valider leur propre dépistage.  
LIRE AUSSI >> Coronavirus : les Américains sans couverture santé dans l'impasse  
D'autre part, un traitement médical contre le nouveau coronavirus pourrait être disponible "d'ici l'été ou le début de l'automne", a affirmé lundi le vice-président américain Mike Pence , qui coordonne la lutte contre l'épidémie aux États-Unis.  Offre limitée. 2 mois pour 1€ sans engagement Je 

In [158]:
seg_auto(TEXT)

1
2
3
4
5
6


[[Un, million, de, tests],
 [d, ', ici, la, fin, de, la, semaine],
 [être, réalisés],
 [pourraient, .],
 [un, traitement],
 [être, disponible],
 [ici, l],
 [ou, l],
 [été, '],
 [d, ', ', automne, ,],
 [selon, Mike, Pence],
 [., ],
 [Et, pourrait],
 [Aux, États-Unis],
 [les, morts],
 [aussi, ,, se, multiplient, .],
 [enregistrés, lundi],
 [Avec, quatre, nouveaux, décès],
 [six, personnes],
 [au, coronavirus, outre-Atlantique],
 [et, la, barre, des, cent, contaminés],
 [a, été, atteinte],
 [,, ont, succombé, ,, .],
 [Selon, le, New, York, Times],
 [l', administration, Trump],
 [un, million, de, tests],
 [d, ', ici, la, fin, de, la, semaine],
 [être, réalisés],
 [qu, ', pourraient],
 [,, a, alors, annoncé, .],
 [Des, compagnies, privées, et, des, laboratoires, académiques],
 [leur, propre, dépistage],
 [et, valider],
 [à, développer],
 [.,  ],
 [ont, été, poussés],
 [LIRE, AUSSI],
 [>, >],
 [les, Américains, sans, couverture, santé, dans, l', impasse,  ],
 [Coronavirus, :],
 [D', autre, p